# HER2 One Scanner - Hamamatsu2

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Hamamatsu2/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, shuffle=True)
print(kf.get_n_splits(y_cat))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=None, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 2s 9ms/step - loss: 1.0456 - acc: 0.5228 - val_loss: 0.9737 - val_acc: 0.5918
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.7483 - acc: 0.6058 - val_loss: 0.7874 - val_acc: 0.6327
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7886 - acc: 0.7137 - val_loss: 0.6613 - val_acc: 0.7347
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6416 - acc: 0.7925 - val_loss: 1.6998 - val_acc: 0.5306
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9822 - acc: 0.6183 - val_loss: 0.6601 - val_acc: 0.7347
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7666 - acc: 0.7676 - val_loss: 0.5837 - val_acc: 0.8367
Epoch 7/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5187 - acc: 0.8257 - val_loss: 0.5000 - val_acc: 0.8367
Epoch 8/1

241/241 [==============================] - 0s 2ms/step - loss: 0.3715 - acc: 0.8631 - val_loss: 0.2562 - val_acc: 0.8776
Epoch 62/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.4353 - acc: 0.8589 - val_loss: 0.2637 - val_acc: 0.8776
Epoch 63/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2395 - acc: 0.9087 - val_loss: 0.3393 - val_acc: 0.8776
Epoch 64/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1929 - acc: 0.9087 - val_loss: 0.2868 - val_acc: 0.8571
Epoch 65/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5804 - acc: 0.8174 - val_loss: 0.2422 - val_acc: 0.8776
Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2306 - acc: 0.9129 - val_loss: 0.2102 - val_acc: 0.8776
Epoch 67/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1938 - acc: 0.9046 - val_loss: 0.5573 - val_acc: 0.8776
Epoch 68/1000
241/241 [==============================] - 0s 2ms/step 

241/241 [==============================] - 0s 2ms/step - loss: 0.0782 - acc: 0.9627 - val_loss: 0.0907 - val_acc: 0.9796
Epoch 122/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9094 - acc: 0.8589 - val_loss: 0.0595 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0809 - acc: 0.9668 - val_loss: 0.0441 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0532 - acc: 0.9793 - val_loss: 0.0637 - val_acc: 0.9592
Epoch 125/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1052 - acc: 0.9419 - val_loss: 0.0364 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0885 - acc: 0.9544 - val_loss: 0.0305 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0684 - acc: 0.9668 - val_loss: 4.0872 - val_acc: 0.5306
Epoch 128/1000
241/241 [==============================] - 0s 2m

241/241 [==============================] - 0s 2ms/step - loss: 0.0290 - acc: 0.9959 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 182/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0396 - acc: 0.9876 - val_loss: 0.0367 - val_acc: 0.9796
Epoch 183/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3206 - acc: 0.9253 - val_loss: 0.0076 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0452 - acc: 0.9876 - val_loss: 0.0133 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0628 - acc: 0.9834 - val_loss: 0.0107 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0636 - acc: 0.9751 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0156 - acc: 1.0000 - val_loss: 0.0154 - val_acc: 1.0000
Epoch 188/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.4555 - acc: 0.8465 - val_loss: 0.2954 - val_acc: 0.9167
Epoch 15/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6222 - acc: 0.8008 - val_loss: 0.3087 - val_acc: 0.9167
Epoch 16/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4313 - acc: 0.8506 - val_loss: 0.3236 - val_acc: 0.9167
Epoch 17/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7097 - acc: 0.7469 - val_loss: 0.2841 - val_acc: 0.9583
Epoch 18/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5179 - acc: 0.8340 - val_loss: 0.2706 - val_acc: 0.9167
Epoch 19/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4389 - acc: 0.8382 - val_loss: 0.1858 - val_acc: 0.9583
Epoch 20/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5013 - acc: 0.8216 - val_loss: 0.6606 - val_acc: 0.7083
Epoch 21/1000
241/241 [==============================] - 0s 1ms/step 

...

241/241 [==============================] - 0s 1ms/step - loss: 0.2000 - acc: 0.9170 - val_loss: 0.1920 - val_acc: 0.9167
Epoch 74/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1448 - acc: 0.9502 - val_loss: 0.1140 - val_acc: 1.0000
Epoch 75/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1335 - acc: 0.9336 - val_loss: 0.4129 - val_acc: 0.7917
Epoch 76/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3174 - acc: 0.8672 - val_loss: 0.0852 - val_acc: 0.9167
Epoch 77/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1865 - acc: 0.9212 - val_loss: 0.0797 - val_acc: 1.0000
Epoch 78/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1027 - acc: 0.9627 - val_loss: 1.3971 - val_acc: 0.4167
Epoch 79/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4119 - acc: 0.8548 - val_loss: 0.1034 - val_acc: 1.0000
Epoch 80/1000
241/241 [==============================] - 0s 2ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.1035 - acc: 0.9585 - val_loss: 1.4561 - val_acc: 0.7083
Epoch 134/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2699 - acc: 0.9461 - val_loss: 0.0166 - val_acc: 1.0000
Epoch 135/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0205 - acc: 0.9959 - val_loss: 0.0085 - val_acc: 1.0000
Epoch 136/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0330 - acc: 0.9917 - val_loss: 0.0110 - val_acc: 1.0000
Epoch 137/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0169 - acc: 0.9959 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 138/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0209 - acc: 0.9959 - val_loss: 0.0054 - val_acc: 1.0000
Epoch 139/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0409 - acc: 0.9876 - val_loss: 0.1038 - val_acc: 0.9583
Epoch 140/1000
241/241 [==============================] - 0s 1m

Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9589 - acc: 0.5436 - val_loss: 0.8287 - val_acc: 0.7500
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8508 - acc: 0.6805 - val_loss: 0.9513 - val_acc: 0.4583
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7949 - acc: 0.6763 - val_loss: 0.8350 - val_acc: 0.6042
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6570 - acc: 0.7635 - val_loss: 0.5386 - val_acc: 0.7917
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6474 - acc: 0.7718 - val_loss: 1.0011 - val_acc: 0.4375
Epoch 8/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7184 - acc: 0.7593 - val_loss: 0.5199 - val_acc: 0.8333
Epoch 9/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6049 - acc: 0.7967 - val_loss: 1.2464 - val_acc: 0.6250
Epoch 10/1000
241/241 [==============================] - 0s 2m

Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1790 - acc: 0.9004 - val_loss: 0.1537 - val_acc: 0.9167
Epoch 65/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5946 - acc: 0.8008 - val_loss: 0.1351 - val_acc: 0.9167
Epoch 66/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2103 - acc: 0.8921 - val_loss: 0.1183 - val_acc: 0.9167
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2171 - acc: 0.8963 - val_loss: 0.1135 - val_acc: 0.9167
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2077 - acc: 0.9087 - val_loss: 0.1073 - val_acc: 0.9167
Epoch 69/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6619 - acc: 0.8340 - val_loss: 0.1638 - val_acc: 0.9167
Epoch 70/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2298 - acc: 0.8880 - val_loss: 0.1042 - val_acc: 0.9167
Epoch 71/1000
241/241 [==============================] 

241/241 [==============================] - 0s 1ms/step - loss: 0.6185 - acc: 0.8382 - val_loss: 0.0089 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0553 - acc: 0.9793 - val_loss: 0.0067 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0530 - acc: 0.9834 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0560 - acc: 0.9710 - val_loss: 0.0053 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0558 - acc: 0.9751 - val_loss: 0.0032 - val_acc: 1.0000
Epoch 129/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0805 - acc: 0.9627 - val_loss: 0.0102 - val_acc: 1.0000
Epoch 130/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0528 - acc: 0.9793 - val_loss: 0.0200 - val_acc: 1.0000
Epoch 131/1000
241/241 [==============================] - 0s 1m